In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate as sci
import time
from getdist import plots, MCSamples
import time

import tensorflow as tf
ds = tf.contrib.distributions

%matplotlib inline
dpi = 200

/home/mauricio/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# saved things directory
direc = '/home/mauricio/Documents/Uni/Intro_2/' + 'gal.txt'

# Carga de datos
redshift = np.genfromtxt('gal.txt', usecols=(1))
mu_obs = np.genfromtxt('gal.txt', usecols=(2)) # m - M
cov = np.genfromtxt('gal.txt', usecols=(3))

p = np.argsort(redshift)
redshift = redshift[p].astype(np.float32)
mu_obs = mu_obs[p]
cov = cov[p]
cov = np.diag(cov)

In [3]:
tf.reset_default_graph()

def f1(theta, z, omk):
    zc = np.copy(z)
    zc = np.insert(zc, 0, 0)
    dz = zc[1:] - zc[:-1]
    E = EHubble(theta, z)[0]
    I = tf.cumsum(dz/(E + 1e-300), axis=1)
    o_k_s = tf.reshape(tf.sqrt(abs(omk)), [batch_size, 1])
    return (1 + z)*tf.sinh(o_k_s*I)/(o_k_s + 1e-300)


def f2(theta, z, omk):
    zc = np.copy(z)
    zc = np.insert(zc, 0, 0)
    dz = zc[1:] - zc[:-1]
    E = EHubble(theta, z)[0]
    I = tf.cumsum(dz/(E + 1e-300), axis=1)
    o_k_s = tf.reshape(tf.sqrt(abs(omk)), [batch_size, 1])
    return (1 + z)*tf.sin(o_k_s*I)/(o_k_s + 1e-300)


def f3(theta, z, omk):
    zc = np.copy(z)
    zc = np.insert(zc, 0, 0)
    dz = zc[1:] - zc[:-1]
    E = EHubble(theta, z)[0]
    I = tf.cumsum(dz/(E + 1e-300), axis=1)
    return (1 + z)*I


def EHubble(theta, z): # parametro de hubble
    """
    theta: parameter space state.
    z: redshift.
    bs: batch size.
    """
    bs = batch_size
    om0 = theta[:, 0]
    ol = theta[:, 1]
    w = theta[:, 2]
    ts = tf.shape(theta)
    zz = np.tile(z, (bs, 1))
    arg = tf.reshape(om0, [ts[0], 1])*(1 + z)**3 + tf.reshape((1 - om0 - ol), [ts[0], 1])*(1 + z)**2 + tf.reshape(ol, [ts[0], 1])*(1 + z)**(3*(1 + tf.reshape(w, [ts[0], 1])))
    EE = tf.sqrt(arg)
    return EE, arg


def modelo(theta, z):    
    om0 = theta[:, 0]
    ol = theta[:, 1]
    w = theta[:, 2]    
    omega_k = 1 - om0 - ol
    sig = tf.sign(omega_k)
    may = tf.reshape(1 + tf.sign(sig - 1), [batch_size, 1])
    men = tf.reshape(1 - tf.abs(sig), [batch_size, 1])
    eq = tf.reshape(1 - tf.sign(sig + 1), [batch_size, 1])    
    dl = may*f1(theta, z, omega_k) + eq*f3(theta, z, omega_k) + men*f2(theta, z, omega_k)
    # integral
    dist = 5*tf.log(dl + 1e-300)/np.log(10)
    return dist


class potential:
    def __init__(self, dat, sigma, z):
        self.data = dat
        self.cov = sigma
        self.z = z
    
    def value(self, theta):
        self.mod = modelo(theta, self.z)
        self.u = - likelihood(self.mod, self.data, self.cov) #- prior(theta, ndim) 
        return self.u
    
    def grad(self, theta):
        self.mod = modelo(theta, self.z)
        self.u = - likelihood(self.mod, self.data, self.cov)
        self.gradient = tf.gradients(self.u, theta)
        return self.gradient


def likelihood(mod, dat, sigma): # retorna escalar, log(L)
    """Log likelihood
    mod: tf.tensor with model results
    dat: numpy array of data
    sigma: numpy array of covariance"""
    sig = tf.cast(tf.diag_part(sigma), tf.float32)
    L = -0.5*chi2(mod, dat, sigma)[0]  + tf.reduce_sum(-0.5*tf.log(2*np.pi*sig**2))
    return L


def chi2(mod, dat, sigma):
    dat1 = np.tile(dat, (batch_size, 1))
    sig = tf.cast(tf.diag_part(sigma), tf.float32)
    sig1 = np.tile(sig, (batch_size, 1))
    AA = tf.reduce_sum(tf.square((dat1 - mod)/sig))
    BB = tf.reduce_sum((dat1 - mod)/tf.square(sig))
    CC = tf.reduce_sum(1/tf.square(sig))
    chi = AA - (BB**2)/CC
    return chi, BB/CC


In [4]:
class prior:
    def __init__(self, name, low=None, high=None, mean=None, cov=None):
        if name=='uniform':
            self.u = tf.distributions.Uniform(low=low, high=high)
        elif name=='normal':
            self.u = tf.contrib.distributions.MultivariateNormalFullCovariance(loc=mean, covariance_matrix=cov)
    
    def get_samples(self, n):
        return self.u.sample(sample_shape=(n))
    
    def get_pdf(self, value):
        return self.u.prob(value)
    
    def get_log_pdf(self, value):
        return self.u.log_prob(value)

In [5]:
class Leapfrog:
    def __init__(self, U, m, ndim, nnx, nnv, e, b):
        """
        U: potential energy function
        m: # of leapfrog steps
        ndim: # of dimensions
        nnx: neural network of x's
        nnv: neural network of v's
        e: leapfrog step parameter
        b: batch size
        """
        self.U = U
        self.m = m
        self.ndim = ndim
        self.nnx = nnx
        self.nnv = nnv
        self.e = e
        self.b = b
        self.x = tf.Variable(tf.random_normal([b, ndim]))
        self.v = tf.Variable(tf.random_normal([b, ndim]))
        self.t = tf.Variable(tf.random_normal([b, 2]))
        
    def direction(self):
        """Samples a new direction"""
        self.d = np.random.choice([-1, 1])
        
    def for_dyn(self):
        """One step of forward dynamics d=1.
        """
        # remember S, Q, T update in each sub iteration
        """
        self.v = self.v*tf.exp(0.5*self.nnv.S*self.e) - 0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + self.nnv.T)
        self.x = self.x*tf.exp(self.e*self.nnx.S) + self.e*(self.v*tf.exp(self.e*self.nnx.Q) + self.nnx.T)
        self.x = tf.squeeze(self.x)
        self.v = tf.squeeze(self.v)        
        self.v = self.v*tf.exp(0.5*self.nnv.S*self.e) - 0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + self.nnv.T)
        """
        tf.assign(self.v, 
                  tf.squeeze(self.v*tf.exp(0.5*self.nnv.S*self.e) - 
                             0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + 
                                         self.nnv.T)))
        tf.assign(self.x, 
                  tf.squeeze(self.x*tf.exp(self.e*self.nnx.S) + 
                             self.e*(self.v*tf.exp(self.e*self.nnx.Q) + self.nnx.T)))      
        tf.assign(self.v, 
                  tf.squeeze(self.v*tf.exp(0.5*self.nnv.S*self.e) - 
                             0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + 
                                         self.nnv.T)))
    
    def back_dyn(self):
        """One step of backward dynamics d=-1.
        """
        """
        self.v = self.v*tf.exp(- 0.5*self.nnv.S*self.e) + 0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + self.nnv.T)
        self.x = self.x*tf.exp(- self.e*self.nnx.S) - self.e*(self.v*tf.exp(self.e*self.nnx.Q) + self.nnx.T)
        self.x = tf.squeeze(self.x)
        self.v = tf.squeeze(self.v) 
        self.v = self.v*tf.exp(- 0.5*self.nnv.S*self.e) + 0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + self.nnv.T)
        """
        tf.assign(self.v, 
                  tf.squeeze(self.v*tf.exp(- 0.5*self.nnv.S*self.e) + 
                             0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + 
                                         self.nnv.T)))
        tf.assign(self.x, 
                  tf.squeeze(self.x*tf.exp(- self.e*self.nnx.S) - self.e*(self.v*tf.exp(self.e*self.nnx.Q) + self.nnx.T)))
        tf.assign(self.v, 
                  tf.squeeze(self.v*tf.exp(- 0.5*self.nnv.S*self.e) + 
                             0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) + 
                                         self.nnv.T)))
        
    def dyn(self):
        """m steps dynamics"""
        #self.direction()
        for i in range(self.m):
            #self.mask = np.random.choice([1, 0], size=(self.b, self.ndim), p=[0.5, 0.5])
            #self.nmask = np.ones((self.b, self.ndim)) - self.mask
            # update outputs of the neural network
            self.time_encoding(i) # updates time
            self.nnx.model(self.x, self.v, self.t)
            self.nnv.model(self.x, self.v, self.t)
            if self.d==1:
                self.for_dyn()
            elif self.d==-1:
                self.back_dyn()
                
                
    def time_encoding(self, mi):
        """Encodes time.
        mi: int, actual leapfrog step.
        """
        arg = 2*np.pi*mi/self.m
        val = np.array([np.cos(arg), np.sin(arg)])
        val = np.tile(val, (self.b, 1))
        val = tf.convert_to_tensor(val)
        tf.assign(self.t, tf.cast(val, tf.float32))
        
        
    def resample(self):
        """Resamples velocity and direction"""
        tf.assign(self.v, tf.random_normal(shape=(self.b, self.ndim), 
                                           mean=0.0, stddev=1.0, dtype=tf.float32))
        self.direction()
        
        
    def flip(self):
        """Flip direction"""
        self.d *= -1      
    
    def sampling(self):
        """Sampling operation"""
        self.direction()
        self.dyn()
        self.flip()
    
    def update_state(self, x0):
        tf.assign(self.x, x0)

In [6]:
class L2HMC:
    def __init__(self, U, prior, n, b, m, lr, sc, reg, lfp, lfq, x0):
        """
        U: energy function
        prior: prior distribution
        n: # iterations
        b: batch size
        m: leapfrog steps
        lr: learning rate
        sc: scale parameter
        reg: regularization parameter
        lfp: inital samples leapfrog object
        lfq: batch samples leapfrog object
        x0: initial position in parameter space
        """
        self.U = U
        self.prior = prior
        self.n = n
        self.b = b
        self.m = m
        self.lr = lr
        self.sc = sc
        self.reg = reg
        self.init_samples = self.prior.get_samples(b)
        self.lfp = lfp
        self.lfq = lfq
        self.X = x0 # ep
        self.optimizer = tf.train.AdamOptimizer(lr)
    
        
    def distance(self):
        self.dp = tf.reduce_sum((self.X - self.X1)**2, axis=1)
        self.dq = tf.reduce_sum((self.Xq - self.xq)**2, axis=1)
    
    def acceptance(self):
        pass
        
    def loss(self, d): 
        return self.sc**2/d - d/self.sc**2
        
    def Loss(self):
        self.distance()
        self.acceptance()
        self.L0ss = (tf.reduce_mean(self.loss(self.dp)) 
                     + tf.reduce_mean(self.reg*self.loss(self.dq)))
        
    def train(self):
        self.Loss()
        self.optimizer.minimize(self.L0ss)
        
    def Run(self):
        for i in range(self.n):
            print(i)
            self.xq = self.prior.get_samples(self.b) # from prior distribution eq
            # first updates samples state 
            self.lfp.update_state(self.X) # now leapfrog knows the state to evolve
            self.lfp.resample()
            self.lfp.sampling()
            self.X1 = self.lfp.x
            # now updates batch samples state
            self.lfq.update_state(self.xq) # now leapfrog knows the state to evolve
            self.lfq.resample()
            self.lfq.sampling()
            self.Xq = self.lfq.x
            # acceptance
            self.train()
            

In [7]:
class MLP:
    def __init__(self, ndim, n1, n2, ls, lq):
        """
        ndim: # of dimensions
        n1: # of neurons of layer 1
        ls: output parameter
        lq: output parameter
        """
        self.W1 = tf.Variable(tf.random_normal([ndim, n1]))
        self.W2 = tf.Variable(tf.random_normal([ndim, n1]))
        self.W3 = tf.Variable(tf.random_normal([2, n1])) # time encoding
        self.W4 = tf.Variable(tf.random_normal([n1, n2]))
        self.Ws = tf.Variable(tf.random_normal([n2, 1]))
        self.Wq = tf.Variable(tf.random_normal([n2, 1]))
        self.Wt = tf.Variable(tf.random_normal([n1, 1]))
        self.b1 = tf.Variable(tf.random_normal([n1]))
        self.b2 = tf.Variable(tf.random_normal([n2]))
        self.bs = tf.Variable(tf.random_normal([ndim]))
        self.bq = tf.Variable(tf.random_normal([ndim]))
        self.bt = tf.Variable(tf.random_normal([ndim]))
        self.ls = ls
        self.lq = lq
    
    def model(self, x, v, t):
        h1 = tf.matmul(x, self.W1) + tf.matmul(v, self.W2) + tf.matmul(t, self.W3) + self.b1
        h1 = tf.nn.relu(h1)
        h2 = tf.matmul(h1, self.W4) + self.b2
        h2 = tf.nn.relu(h2)
        self.S = tf.tanh(tf.matmul(h2, self.Ws) + self.bs)
        self.Q = tf.tanh(tf.matmul(h2, self.Wq) + self.bq)
        self.T = tf.matmul(h2, self.Wt) + self.bt
    
    
   

In [8]:
low = [0., 0., -6.]
high = [1., 1., 1/3]
global batch_size
batch_size = 2

x1 = np.array([[0.5, 0.4, -0.5], [0.5, 0.4, -0.5]])
x1 = tf.convert_to_tensor(x1, dtype=tf.float32)

v1 = np.array([[0.3, 0.4, -1.5], [0.3, 0.4, -1.5]])
v1 = tf.convert_to_tensor(v1, dtype=tf.float32)

t1 = np.zeros((2, 2))
t1 = tf.convert_to_tensor(t1, dtype=tf.float32)

a1 = np.array([1, 2, 3])
a1 = tf.convert_to_tensor(a1, dtype=tf.float32)

In [9]:
pot = potential(mu_obs, cov, redshift)
pr = prior(name='uniform', low=low, high=high)
mlpx = MLP(ndim=3, n1=10, n2=10, ls=1, lq=1)
mlpv = MLP(ndim=3, n1=10, n2=10, ls=1, lq=1)
mlpx.model(x1, v1, t1)
mlpv.model(x1, v1, t1)
lfp = Leapfrog(U=pot, m=2, ndim=3, nnx=mlpx, nnv=mlpv, e=1e-3, b=batch_size)
lfq = Leapfrog(U=pot, m=2, ndim=3, nnx=mlpx, nnv=mlpv, e=1e-3, b=batch_size)
l2 = L2HMC(pot, pr, 2, batch_size, 2, 1e-3, 1e-1, 1e-1, lfp, lfq, x1)

In [15]:
l2.Run()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(l2.lfp.nnx.W1))
    print(sess.run(l2.lfp.nnv.W1))

0
1
[[-0.1123492  -0.5204956  -0.16787426 -0.9697638  -0.5113813  -0.4466553
   0.6632534  -0.20490135  1.4734832   0.447116  ]
 [-0.27790308  0.12932882 -1.6955649  -1.3449037   0.6442632   1.5062255
  -1.7222717  -0.61809576  3.204022   -0.6150305 ]
 [ 0.58014315 -0.23208265  0.6887078   0.08456226 -2.0291688  -0.06854253
  -0.29231626 -1.8247484   0.740344    1.8968562 ]]
[[-0.8560113  -0.6002867  -1.1473316   0.9116065  -0.17145513 -0.29820508
  -0.5367889   0.74173576 -1.762843    2.9269288 ]
 [-0.5501381   0.24169864 -0.37297893  1.480622    0.67826414 -0.1811315
   0.24152836 -0.05104424 -0.22892423 -1.9216114 ]
 [ 0.62063205 -0.28846878  2.134821   -0.0208789   0.31769362  0.4780832
   0.15362573 -0.5068271   0.20517947  1.6152414 ]]


In [3]:
redshift

array([0.015     , 0.015     , 0.015     , 0.01502704, 0.0151    ,
       0.015166  , 0.0152    , 0.0152    , 0.0153    , 0.0154363 ,
       0.016     , 0.016     , 0.0163    , 0.016321  , 0.01634564,
       0.01645   , 0.016559  , 0.01673   , 0.016743  , 0.016991  ,
       0.017173  , 0.017227  , 0.0173    , 0.0173    , 0.017605  ,
       0.01793128, 0.01831523, 0.0187    , 0.0189    , 0.0192    ,
       0.0192    , 0.019264  , 0.0195    , 0.019599  , 0.02037472,
       0.0205    , 0.0208    , 0.0209    , 0.0211    , 0.0212    ,
       0.0215    , 0.021793  , 0.0219    , 0.02198001, 0.0221    ,
       0.0221    , 0.0224    , 0.0229    , 0.0229    , 0.02297117,
       0.023     , 0.023208  , 0.0233    , 0.0233    , 0.023536  ,
       0.0239    , 0.023953  , 0.024     , 0.0241853 , 0.0242    ,
       0.024314  , 0.024525  , 0.0247    , 0.0248    , 0.0249    ,
       0.0251    , 0.0255    , 0.0256    , 0.0259    , 0.026     ,
       0.026038  , 0.0261    , 0.026489  , 0.0266    , 0.02680